In [5]:
from flask import Flask
from flask import Blueprint, request
from werkzeug.utils import secure_filename
import numpy as np
import tensorflow as tf
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS, ALSModel
from scipy.spatial.distance import cosine
import joblib
from tensorflow.keras.applications import MobileNetV2
import json
import uuid

In [6]:
img_shape = (160, 160, 3)

# 사전 학습된 MobileNetV2 모델을 base_model로 저장
base_model = MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')
# ***dropout등 튜닝을 해도 될거같다!
# include_top: 네트워크의 최상단에 완전연결 레이어를 넣을지 여부
# 참고로 마지막 분류 계층 ("상단")은 피쳐 추출에 그리 유용하지 않으므로 include_top=False를 지정해 맨 위에 분류 계층을 제외
# weights: 'imagenet' (ImageNet에 대한 선행 학습)

# tf.keras.layers.GlobalAveragePooling2D 레이어를 사용하여 피처를 이미지 당 하나의 1280 요소 벡터로 변환
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# base_model에 global_average_layer 쌓기
neural_network = tf.keras.Sequential([
  base_model,
  global_average_layer,
])
# joblib.dump(neural_network,'./model/mobilenetv2.pkl')

9420800/9406464 [==============================] - 1s 0us/step


In [7]:
spark = SparkSession.builder.master('local').appName('all').getOrCreate()

In [8]:
#######모델 불러오기 (dnn feature뽑아오기,als 해시태그 추천)
# model_path='/content/drive/MyDrive/insta_crawling/Hashtag_Team16/modeling (1)/mobilenetv2.pkl'
# neural_network = joblib.load(model_path)
als_model = ALSModel.load('modeling/als')

#######데이터 불러오기 (pics 이미지의 피처 데이터프레임,hashtags_df 모든해시태그, recommender_df 추천df)
recommender_df=joblib.load('recommender_df_0424_1.pkl')
hashtags_df=joblib.load('hashtags_df_0424_1.pkl')

In [10]:
def prepare_image(img_path, height=160, width=160):
    # 신경망에 맞게 이미지를 다운 샘플링 및 스케일링
    img = tf.io.read_file(img_path) # 불러(읽어)오기
    #img = tf.image.decode_jpeg(img) # [height, width, num_channels]인 3차원 배열을 반환
    img = tf.image.decode_image(img)
    img = tf.cast(img, tf.float32) # 정수형으로 바꾼경우 소수점을 버린다 boolean일때는 True면 1, False면 0을 출력
    img = (img/127.5) - 1
    img = tf.image.resize(img, (height, width))
    # 컬러 이미지의 차원에 맞게 회색조 이미지 형태변경
    if img.shape != (160, 160, 3):
        img = tf.concat([img, img, img], axis=2)
    return img

def extract_features(image, neural_network):
    # input받은 이미지를 1280개의 deep feature들로 구성된 벡터로 반환
    image_np = image.numpy() # numpy형태로 변환
    images_np = np.expand_dims(image_np, axis=0) # 차원추가([]를 씌워준다)
    deep_features = neural_network.predict(images_np)[0]
    return deep_features

# 코사인 유사성에 기반한 K개의 최근접이웃을 찾는 함수 
def find_neighbor_vectors(image_path, k=5, recommender_df=recommender_df):
    # 비슷한 이미지에 대한 img_features(이미지 피쳐, 즉 사용자 벡터)를 찾는다.
    prep_image = prepare_image(image_path)
    pics = extract_features(prep_image, neural_network)
    rdf = recommender_df.copy()
    rdf['dist'] = rdf['deep_features'].apply(lambda x: cosine(x, pics))
    rdf = rdf.sort_values(by='dist')
    return rdf.head(k)
  

def generate_hashtags(image_path):
    fnv = find_neighbor_vectors(image_path, k=5, recommender_df=recommender_df)
    # 코사인 유사성에 기반하여 5개의 사용자 벡터의 평균을 구한다.
    features = []
    for item in fnv.features.values:
        features.append(item)

    avg_features = np.mean(np.asarray(features), axis=0)
    
    hashtag_features = als_model.itemFactors.toPandas()

    # 앞서 구한 이미지(사용자) 피쳐의 평균, 즉 avg_features을 hashtag_features와 dot product하여 새로운 dot_product열 생성
    hashtag_features['dot_product'] = hashtag_features['features'].apply(lambda x: np.asarray(x).dot(avg_features))
    
    # 가장 높은 dot product를 가진 해시태그 상위 10개 추출
    final_recs = hashtag_features.sort_values(by='dot_product', ascending=False).head(50)

    # hastag_id로 hashtag 찾아서 output에 저장
    output = []
    for hashtag_id in final_recs.id.values:
        output.append(hashtags_df.iloc[hashtag_id]['hashtag'])
    return output

In [1]:
from flask import Flask
from flask import Blueprint, request
from werkzeug.utils import secure_filename

app = Flask(__name__)
# HTTP POST방식으로 전송된 이미지를 저장
@app.route('/', methods=['POST'])
def save_image():
    f = request.files['file']
    secure_name = secure_filename(f.filename)
    unique_name = str(uuid.uuid1())
    print(unique_name + secure_filename(f.filename))
    f.save('save_image/' + unique_name + secure_filename(f.filename))
    recommended_hashtags = generate_hashtags('save_image/' + unique_name + secure_filename(f.filename)) #원래는 이미지 확장자 생각해서 적용해야함.
    hashtags_dict = {'hashtags':recommended_hashtags}
    hashtags_json = json.dumps(hashtags_dict, sort_keys=True)
    return hashtags_json

if __name__ == '__main__':
    app.run(host='0.0.0.0')